**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath("C:/FFmpeg/bin")
import skvideo.io

import cv2
import json
import random as rd
from collections import deque

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten, Dropout


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?



Answer 1:

The function act help the agent take an action given a current state of the environment s. There are several options to choose from and during training we can randomly choose one of them. After multiple iterations of training, the agent finds an optimal option that optimizes the reward function. The algorithm will always choose this action (called exploitation). 
$$ a^{*} = argmax_{a} Q^{\pi} (s , a) $$

However there might be other actions still that are better. Looking for these options is called exploration and the standard way to explore is to introduce an additional term, epsilon. We genereate a  random value and if it is less than epsilon, a random act is chosen. If not, the learned act that optimizes reward is chosen. Epsilon manages the tradeoff between explortion and exploitation.


After we’ve explored all the options and we know for sure the best option, the exploration should turn off (we set train to false)


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 20 # set small when debugging
epochs_test= 20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array contains the rat's position on the board at time t. Every time act is called, all the elements of the array are set to zeros except the position of the rat that is set to 1. The border 2 grids on all four sides are set  to -1.

The board array consists of the reward view of the board on which the rat moves. The cells can be one of the three things; neutral, poisonous or conatin cheese. The cells represents the reward of each type and are respectively 0, -1 and 0.5. Reward distribution is defined in the reset function with binomial law


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [27]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return rd.randint(0,3) #generates one of 0,1,2 or 3 randomly as these are the 
                         #four actions available to the agent
        
        

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [28]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        win = 0
        lose = 0
        game_over = False
        
        state = env.reset()
        
        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [29]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.5/20.0. Average score (-7.5)
Win/lose count 5.5/7.0. Average score (-4.5)
Win/lose count 8.0/14.0. Average score (-5.0)
Win/lose count 5.5/13.0. Average score (-5.625)
Win/lose count 6.0/11.0. Average score (-5.5)
Win/lose count 12.5/15.0. Average score (-5.0)
Win/lose count 7.5/11.0. Average score (-4.785714285714286)
Win/lose count 6.0/7.0. Average score (-4.3125)
Win/lose count 10.5/15.0. Average score (-4.333333333333333)
Win/lose count 8.0/17.0. Average score (-4.8)
Win/lose count 7.0/17.0. Average score (-5.2727272727272725)
Win/lose count 11.5/12.0. Average score (-4.875)
Win/lose count 12.5/23.0. Average score (-5.3076923076923075)
Win/lose count 5.5/10.0. Average score (-5.25)
Win/lose count 9.5/11.0. Average score (-5.0)
Win/lose count 9.0/20.0. Average score (-5.375)
Win/lose count 14.0/22.0. Average score (-5.529411764705882)
Win/lose count 10.0/13.0. Average score (-5.388888888888889)
Win/lose count 10.5/6.0. Average score (-4.868421052631579)
Win/lose co

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




A5) The action-value function can be written as 

Thus \begin{equation} Q^\pi(s,a)=E{p^{\pi}}[\sum{t\leq T}\gamma^{t}r(s{t},a{t})|s{0}=s,a{0}=a] > . \end{equation}

We use the Markov property and define the function as a Bellman equation:

$$\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}$$
One assumes that the optimal policy $\pi^*$ exists, the optimal Q function is defined as:

$$\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \&gt; 
\end{equation*}$$$$\begin{equation*}
Q^*(s,a)= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \&gt; 
\end{equation*}$$$$\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')] \&gt; 
\end{equation*}$$$$\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}$$

Considering mean square, a plausible objective is to minimize the loss:

$$\begin{equation*}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}( ( Q^* (s,a,\theta ) - Q(s,a,\theta)) ^2) 
\end{equation*}$$$$\begin{equation*}
\mathcal{L}(\theta) =  E_{s' \sim \pi^*(.|s,a)} ( ( r(s,a)+\gamma\max_{a'}Q^{*}(s',a', \theta) - Q(s,a,\theta)) ^2) 
\end{equation*}$$$$\begin{equation*}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}$$


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen=self.max_memory) 

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        idx = rd.randint(0,len(self.memory)-1)
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch = 5,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, axis=0)), axis=1).astype(int)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(np.expand_dims(n_s_, axis=0))
            if game_over_:
                target_q[i, a_] = r_
            else:
                target_q[i,a_] = r_ + self.discount * np.amax(self.model.predict(np.expand_dims(n_s_, axis=0)), axis=1)
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        state_size = 50
        
        model = Sequential()
        model.add(Reshape((state_size,), input_shape=(5,5,self.n_state)))
        model.add(Dense(256, activation='relu', input_dim=5*5*self.n_state))
        model.add(Dense(32, activation='relu',))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/020 | Loss 0.0028 | Win/lose count 4.5/7.0 (-2.5)
Epoch 001/020 | Loss 0.0069 | Win/lose count 3.5/0 (3.5)
Epoch 002/020 | Loss 0.0098 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/020 | Loss 0.0169 | Win/lose count 5.0/3.0 (2.0)
Epoch 004/020 | Loss 0.0105 | Win/lose count 1.0/6.0 (-5.0)
Epoch 005/020 | Loss 0.0290 | Win/lose count 3.0/9.0 (-6.0)
Epoch 006/020 | Loss 0.0087 | Win/lose count 6.0/5.0 (1.0)
Epoch 007/020 | Loss 0.0066 | Win/lose count 4.0/4.0 (0.0)
Epoch 008/020 | Loss 0.0153 | Win/lose count 2.5/2.0 (0.5)
Epoch 009/020 | Loss 0.0485 | Win/lose count 9.5/7.0 (2.5)
Epoch 010/020 | Loss 0.0519 | Win/lose count 3.5/1.0 (2.5)
Epoch 011/020 | Loss 0.0134 | Win/lose count 5.0/2.0 (3.0)
Epoch 012/020 | Loss 0.0180 | Win/lose count 8.0/5.0 (3.0)
Epoch 013/020 | Loss 0.0678 | Win/lose count 4.5/3.0 (1.5)
Epoch 014/020 | Loss 0.0077 | Win/lose count 4.5/5.0 (-0.5)
Epoch 015/020 | Loss 0.0060 | Win/lose count 2.0/1.0 (1.0)
Epoch 016/020 | Loss 0.0116 | Win/lose count 7.0/10.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, 2, activation='relu', input_shape=(5,5,self.n_state)))
        model.add(Conv2D(16, 2, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
    
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/020 | Loss 0.0145 | Win/lose count 4.0/5.0 (-1.0)
Epoch 001/020 | Loss 0.0690 | Win/lose count 5.5/2.0 (3.5)
Epoch 002/020 | Loss 0.0155 | Win/lose count 6.5/10.0 (-3.5)
Epoch 003/020 | Loss 0.0072 | Win/lose count 5.5/7.0 (-1.5)
Epoch 004/020 | Loss 0.0032 | Win/lose count 7.5/6.0 (1.5)
Epoch 005/020 | Loss 0.0714 | Win/lose count 2.5/7.0 (-4.5)
Epoch 006/020 | Loss 0.0116 | Win/lose count 11.5/8.0 (3.5)
Epoch 007/020 | Loss 0.0219 | Win/lose count 14.0/4.0 (10.0)
Epoch 008/020 | Loss 0.0084 | Win/lose count 12.0/11.0 (1.0)
Epoch 009/020 | Loss 0.0199 | Win/lose count 7.5/2.0 (5.5)
Epoch 010/020 | Loss 0.0101 | Win/lose count 7.5/2.0 (5.5)
Epoch 011/020 | Loss 0.0077 | Win/lose count 8.0/7.0 (1.0)
Epoch 012/020 | Loss 0.0041 | Win/lose count 5.0/1.0 (4.0)
Epoch 013/020 | Loss 0.0171 | Win/lose count 13.0/6.0 (7.0)
Epoch 014/020 | Loss 0.0037 | Win/lose count 6.0/0 (6.0)
Epoch 015/020 | Loss 0.0077 | Win/lose count 17.0/2.0 (15.0)
Epoch 016/020 | Loss 0.0750 | Win/lose count 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [16]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 0/0. Average score (0.75)
Win/lose count 0.5/0. Average score (0.6666666666666666)
Win/lose count 4.0/0. Average score (1.5)
Win/lose count 2.0/1.0. Average score (1.4)
Win/lose count 4.0/0. Average score (1.8333333333333333)
Win/lose count 0/0. Average score (1.5714285714285714)
Win/lose count 2.0/0. Average score (1.625)
Win/lose count 0.5/0. Average score (1.5)
Win/lose count 2.5/1.0. Average score (1.5)
Win/lose count 0.5/1.0. Average score (1.3181818181818181)
Win/lose count 1.5/1.0. Average score (1.25)
Win/lose count 0/0. Average score (1.1538461538461537)
Win/lose count 2.0/0. Average score (1.2142857142857142)
Win/lose count 2.0/0. Average score (1.2666666666666666)
Win/lose count 1.5/0. Average score (1.28125)
Win/lose count 3.0/0. Average score (1.3823529411764706)
Win/lose count 7.0/2.0. Average score (1.5833333333333333)
Win/lose count 0.5/0. Average score (1.5263157894736843)
Win/lose count 1.5/0. Av

In [17]:
HTML(display_videos('cnn_test10.mp4'))

In [18]:
HTML(display_videos('fc_test10.mp4'))

Answer9) We observe that the agent doesn't explore other areas much and tends to stick to the current board i.e so low exploration rate. However the results given by the CNN are better than the ones given by the FC model.

If we increase the temperature, both models perform better as there are more rewards on the grid (hence the agent is guided better). But we introduce some bias in the model as doing so increases the probability of having cheese and lowers the probility of poisonious cells

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [42]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    epsilon_start = 1.0
    epsilon_end = 0.1
    
    # The epsilon decay schedule
    epsilons = np.linspace(epsilon_start, epsilon_end, epoch)

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        agent.set_epsilon(epsilons[e])
        
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)#we set train to true here

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4       
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))    
                
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        
        #same as before
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        #new argument
        self.malus_position = np.zeros((grid_size,grid_size))
        
        #same as before
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
    
    def get_t(self):
        return self.t
    
    #same as earlier environment
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
        
    #same as earlier environment    
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
    
    #same act function as before for all the 4 possible actions
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        self.t = self.t + 1     
#You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        # 3 "feature" states instead of 2
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time 
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
       
        self.malus_position[self.x, self.y] = 0

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state
    
    
    
    
    
    
    
    
    
        
        
        

In [43]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0341 | Win/lose count 8.5/28.300000000000004 (-19.800000000000004)
Epoch 001/020 | Loss 0.0189 | Win/lose count 5.200000000000001/32.80000000000013 (-27.60000000000013)
Epoch 002/020 | Loss 0.0133 | Win/lose count 4.5/27.000000000000085 (-22.500000000000085)
Epoch 003/020 | Loss 0.0141 | Win/lose count 8.000000000000002/32.60000000000013 (-24.60000000000013)
Epoch 004/020 | Loss 0.0218 | Win/lose count 11.000000000000005/27.100000000000023 (-16.100000000000016)
Epoch 005/020 | Loss 0.0107 | Win/lose count 15.200000000000005/25.700000000000053 (-10.500000000000048)
Epoch 006/020 | Loss 0.0340 | Win/lose count 10.000000000000004/26.30000000000008 (-16.300000000000075)
Epoch 007/020 | Loss 0.0232 | Win/lose count 12.000000000000005/33.800000000000054 (-21.800000000000047)
Epoch 008/020 | Loss 0.0132 | Win/lose count 12.000000000000005/27.900000000000084 (-15.900000000000079)
Epoch 009/020 | Loss 0.0228 | Win/lose count 13.600000000000007/27.600000000000076 (-14.0000

In [44]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 1.5/0. Average score (1.5)
Win/lose count 8.0/0. Average score (4.75)
Win/lose count 4.0/1.0. Average score (4.166666666666667)
Win/lose count 4.5/0. Average score (4.25)
Win/lose count 8.0/0. Average score (5.0)
Win/lose count 0.5/0. Average score (4.25)
Win/lose count 3.0/1.0. Average score (3.9285714285714284)
Win/lose count 4.5/1.0. Average score (3.875)
Win/lose count 4.0/0. Average score (3.888888888888889)
Win/lose count 10.0/0. Average score (4.5)
Win/lose count 3.5/1.0. Average score (4.318181818181818)
Win/lose count 9.5/0. Average score (4.75)
Win/lose count 2.0/0. Average score (4.538461538461538)
Win/lose count 16.5/0. Average score (5.392857142857143)
Win/lose count 4.5/0. Average score (5.333333333333333)
Win/lose count 5.0/1.0. Average score (5.25)
Win/lose count 11.0/0. Average score (5.588235294117647)
Win/lose count 11.0/1.0. Average score (5.833333333333333)
Win/lose count 13.5/0. Average score (6.2368421052631575)
Win/lose count 7.0/0. Average score 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***